# Spider-Man Play Generator

## Imports

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


## Loading data

In [2]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving spiderman_script.txt to spiderman_script.txt


## Reading content from file

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
'Length of text: {} characters'.format(len(text))

'Length of text: 184901 characters'

In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

SERENITY NOW
Written by
Chris McKenna & Erik Sommers
1.
OVER SONY LOGO:
NY1 REPORTER (V.O.)
We come to you now with revelations
about last week’s attack in London.
An anonymous source provided this
video. It shows Quentin Beck, aka
Mysterio


## Econding characters from script

In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [11]:
index = 13

# lets look at how part of our text is encoded
print("Text:", text[-index:])
print("Encoded:", text_to_int(text[-index:]))

Text: THE END
181.
Encoded: [45 33 30  3 30 39 29  2  1 15 22 15 12]


Converting values back to chars

In [12]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[-index:]))

THE END
181.


## Creating training examples


In [13]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [15]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [16]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
SERENITY NOW
Written by
Chris McKenna & Erik Sommers
1.
OVER SONY LOGO:
NY1 REPORTER (V.O.)
We

OUTPUT
ERENITY NOW
Written by
Chris McKenna & Erik Sommers
1.
OVER SONY LOGO:
NY1 REPORTER (V.O.)
We 


EXAMPLE

INPUT
come to you now with revelations
about last week’s attack in London.
An anonymous source provided 

OUTPUT
ome to you now with revelations
about last week’s attack in London.
An anonymous source provided t


In [17]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Building the model


In [53]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           22016     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 86)            88150     
                                                                 
Total params: 5,357,142
Trainable params: 5,357,142
Non-trainable params: 0
_________________________________________________________________


## Creating a loss function

In [20]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In [21]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 4.1582729e-04  1.1267873e-03 -3.4999289e-03 ... -3.5934914e-03
    6.2885182e-04  2.3319998e-03]
  [ 5.1123798e-03 -8.5950037e-04 -2.9191817e-03 ... -5.4203981e-04
    1.7348769e-03 -4.5098057e-03]
  [-1.2837355e-03  3.5828613e-03 -2.2711291e-03 ... -4.9767346e-04
    1.9495132e-03 -4.4042482e-03]
  ...
  [ 5.7322164e-03 -8.9301774e-04 -7.7725290e-03 ...  3.4827625e-03
    4.9948222e-03  3.4134421e-03]
  [ 8.8854097e-03 -3.3415304e-03 -5.2953125e-03 ...  2.8801300e-03
    1.5438300e-03  9.2389155e-03]
  [ 7.2441464e-03 -1.4247068e-03 -6.8907104e-03 ... -2.8459427e-03
    1.8061472e-03  8.5631153e-03]]

 [[-3.6305001e-03  2.3801078e-04  1.3262633e-03 ... -3.4347838e-03
    3.2913426e-03  7.2987778e-03]
  [-2.1777391e-03  1.5644935e-03 -5.0677784e-04 ... -5.5863224e-03
    4.2505693e-03  3.5481320e-03]
  [-2.6912205e-03 -4.4680055e-04 -2.1831181e-03 ... -5.6260410e-03
   -2.6864058e-04 -1.6141696e-03]
  ...
  [-7.3123365e-03  4.5542973e-03  8.3528110e-04 ...  5.5630603e

In [22]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00041583  0.00112679 -0.00349993 ... -0.00359349  0.00062885
   0.002332  ]
 [ 0.00511238 -0.0008595  -0.00291918 ... -0.00054204  0.00173488
  -0.00450981]
 [-0.00128374  0.00358286 -0.00227113 ... -0.00049767  0.00194951
  -0.00440425]
 ...
 [ 0.00573222 -0.00089302 -0.00777253 ...  0.00348276  0.00499482
   0.00341344]
 [ 0.00888541 -0.00334153 -0.00529531 ...  0.00288013  0.00154383
   0.00923892]
 [ 0.00724415 -0.00142471 -0.00689071 ... -0.00284594  0.00180615
   0.00856312]], shape=(100, 86), dtype=float32)


In [23]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

86
tf.Tensor(
[ 4.1582729e-04  1.1267873e-03 -3.4999289e-03 -1.3079775e-03
 -1.7320107e-03 -1.6314104e-03 -1.5126857e-03  3.1048018e-03
 -9.5255236e-05  1.6014897e-03  1.2355731e-05 -3.5431592e-03
 -7.7606121e-05 -5.3844452e-03  6.2133358e-03 -3.0972173e-03
 -3.2018483e-04  3.9916388e-03  9.1167114e-04  4.5815101e-03
  6.0445271e-03  2.1438433e-03  2.1346675e-03  1.7151961e-03
 -4.8274770e-03 -2.7973100e-03 -2.3786416e-03  2.9443244e-03
 -4.6162597e-05  3.8739768e-04  1.2525801e-03  4.8028929e-03
 -4.5617940e-03  2.6781659e-04 -2.9141211e-03 -8.8350354e-03
  2.7020839e-03  1.7967755e-03 -3.6267433e-03 -1.2698912e-04
  5.0790753e-04 -1.3104663e-03  3.3089609e-03 -4.5861150e-04
  7.1563688e-04 -5.6043621e-03  3.6566269e-03 -4.9795448e-03
 -2.7506836e-04  4.0580342e-03 -5.5871634e-03 -3.4922376e-04
 -3.6117589e-04 -1.1426937e-03  3.1968823e-03 -1.1064420e-04
  4.3381800e-04 -4.2654420e-03 -2.1611750e-03 -2.9105439e-03
  2.5432925e-03 -1.0131163e-03  3.2673632e-03 -3.4953286e-03
 -7.367074

In [24]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"é!(iK“0Ix 0SGsga\nY35j8!GJ8,a!bBg1/z9N.v[I\nq1“.]M29à[c?XV!G5S9cM7ée:&]5#D8xé‘’iXQE'Eec\x02'\nLY6!J\rJj7liD"

In [48]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

## Compiling model

In [54]:
model.compile(optimizer='adam', loss=loss)

### Creating checkpoints for future use

In [55]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Training

In [51]:
EPOCHS = 50

In [56]:
history = model.fit(data, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
28/28 [==============================] - 7s 181ms/step - loss: 3.7071
Epoch 2/50
28/28 [==============================] - 6s 179ms/step - loss: 3.1710
Epoch 3/50
28/28 [==============================] - 5s 180ms/step - loss: 2.6219
Epoch 4/50
28/28 [==============================] - 5s 178ms/step - loss: 2.3507
Epoch 5/50
28/28 [==============================] - 5s 178ms/step - loss: 2.1531
Epoch 6/50
28/28 [==============================] - 5s 179ms/step - loss: 1.9889
Epoch 7/50
28/28 [==============================] - 5s 181ms/step - loss: 1.8633
Epoch 8/50
28/28 [==============================] - 5s 178ms/step - loss: 1.7604
Epoch 9/50
28/28 [==============================] - 5s 178ms/step - loss: 1.6784
Epoch 10/50
28/28 [==============================] - 5s 178ms/step - loss: 1.6009
Epoch 11/50
28/28 [==============================] - 5s 178ms/step - loss: 1.5331
Epoch 12/50
28/28 [==============================] - 5s 178ms/step - loss: 1.4706
Epoch 13/50
28/28 [======

## Loading model

In [57]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [58]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate text from model

In [59]:
def generate_text(model, start_string, num_generate=800, temperature=1.0):
  """
  Generate text from model, starting string, characters to generate and temperature.
  temperature: Low temperatures results in more predictable text.
  Higher temperatures results in more surprising text.
  """

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [63]:
starting_string = input("Type a starting string: ")
num_chars = int(input("Type chars to generate: "))
temperature = float(input("Type temperature: "))
print(generate_text(model, starting_string, num_chars, temperature))

Type a starting string: SPIDER-MAN
Type chars to generate: 200
Type temperature: 0.75
SPIDER-MAN
I have to try.
Dr. Strange stares back, equal parts furious and concerned,
as the PORTAL CLUSES behind him.
RAIMI-VERSE PETER (CONT'D)
Oh, that’s good.
Webb-Verse Peter sets Raimi-Verse Peter’s
